#### Imports

In [25]:
import os
import re
import skrf as rf
import pandas as pd

#### Paths

In [26]:
script_directory = os.getcwd()#obtem diretório atual
s2ps_path = os.path.join(script_directory, 'oil_data','oil_data') #cria string caminho para todos os .s2p

#### Regex = hell

In [27]:
regex = r'SAW_06_02_2024_OIL_MILHO_(\d+)\.S2P'
#SAW_06_02_2024_OIL_GIRASSOL_01.S2P

In [28]:
def read_s2p_vegetal_data(regex, path):
    dfs = []#lista para ocncatenar todos os dicts de cada medição 
    for filename in os.listdir(path):
        match = re.match(regex, filename)
        if match:
            #obtem informação do nome do arquivo
            oil_name = "CORN"
            measurement_number = int(match.group(1))
            
            #carrega arquivo
            path_to_oil = os.path.join(path, filename)
            s2p_file = rf.Network(path_to_oil)
            
            #extrai frequência e parâmetros S do objeto Network
            frequency = s2p_file.f
            s_parameters = s2p_file.s_db
            
            #Dataframe com informações do objeto Network
            individual_oil_dict = {
                'OIL NAME': oil_name,
                'MEASUREMENT NUMBER': measurement_number,
                'FREQUENCY (MHZ)':frequency/1000000,
                'S11 (db)' : s_parameters[:,0,0],
                'S21 (db)' : s_parameters[:,1,0],
                'S12 (db)' : s_parameters[:,0,1],
                'S22 (db)' : s_parameters[:,1,1],
            }
            df = pd.DataFrame.from_dict(individual_oil_dict)
            dfs.append(df)
            
    dataframe = pd.concat(dfs, ignore_index=True)
    return dataframe

#### Read Dataframes

In [29]:
df_vegetal = read_s2p_vegetal_data(regex, s2ps_path)

In [30]:
df_vegetal.info

<bound method DataFrame.info of       OIL NAME  MEASUREMENT NUMBER  FREQUENCY (MHZ)  S11 (db)  S21 (db)  \
0         CORN                   1       110.000000 -0.977038 -79.53358   
1         CORN                   1       110.002000 -0.977490 -77.64947   
2         CORN                   1       110.004000 -0.975435 -81.50738   
3         CORN                   1       110.006001 -0.977087 -78.02262   
4         CORN                   1       110.008001 -0.978815 -78.01198   
...        ...                 ...              ...       ...       ...   
29995     CORN                   3       129.991999 -1.356632 -73.05645   
29996     CORN                   3       129.993999 -1.357346 -71.35054   
29997     CORN                   3       129.996000 -1.359289 -69.55155   
29998     CORN                   3       129.998000 -1.358120 -68.45490   
29999     CORN                   3       130.000000 -1.358705 -68.83554   

       S12 (db)  S22 (db)  
0     -78.71073 -1.020574  
1     -76.3

In [31]:
#assegura somente óleo de girassol lido
for name in df_vegetal["OIL NAME"]:
    if name != "CORN":
        print("Diferente de milho")

In [32]:
#cria .csv caso precise no futuro.
path_save_csv = os.path.join(script_directory, 'oil_data','oil_data','CSVs')
if not os.path.exists(path_save_csv):
   os.makedirs(path_save_csv)
df_vegetal.to_csv(path_save_csv + '/corn_measurements.csv')

In [33]:
def average_measurements(dataframe):
    #agrupa valores por frequência e calcula média do que interessa
    return dataframe.groupby('FREQUENCY (MHZ)', as_index=False)['S21 (db)'].mean()

In [34]:
df_corn_s21 = average_measurements(df_vegetal)

In [35]:
df_corn_s21.info


<bound method DataFrame.info of       FREQUENCY (MHZ)   S21 (db)
0          110.000000 -77.934123
1          110.002000 -78.218783
2          110.004000 -79.051460
3          110.006001 -80.088083
4          110.008001 -81.159780
...               ...        ...
9995       129.991999 -70.879330
9996       129.993999 -70.960103
9997       129.996000 -70.076650
9998       129.998000 -67.898483
9999       130.000000 -67.998407

[10000 rows x 2 columns]>

In [36]:
#cria .csv caso precise no futuro.
path_save_csv_processed = os.path.join(script_directory, 'oil_data','oil_data','CSVs','processed')
if not os.path.exists(path_save_csv_processed):
   os.makedirs(path_save_csv_processed)
df_vegetal.to_csv(path_save_csv_processed + '/corn_measurements_processed.csv')